# Books and Wings

## **Data Collection**

### Web Scraping Amazon
##### At this step we are going to collect data directly from Amazon. Making this step will demand analyze the HTML from Amazon site. Note that this HTMLs are constantly changing, therefore this code can be old. The most important is understand and adapt to the day you are using.

In [ ]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
from time import sleep

import requests
from bs4 import BeautifulSoup

headers = {
    'Host': 'www.amazon.com',
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:70.0) Gecko/20100101 Firefox/70.0',
    'Accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,*/*;q=0.8',
    'Accept-Language': 'en-US,en;q=0.5',
    'Accept-Encoding': 'gzip, deflate, br',
    'Connection': 'keep-alive',
    'Upgrade-Insecure-Requests': '1',
    'TE': 'Trailers'
}

alls = []

for k in range(1,3):

  URL = "https://www.amazon.com/best-sellers-books-Amazon/zgbs/books/ref=zg_bs_pg_"+str(k)+"?_encoding=UTF8&pg="+str(k)
  page = requests.get(URL, headers=headers)
  soup = BeautifulSoup(page.content, "html.parser")

  i=0
  book = True

  while(book!=None):

    all = []

    try:

      book = soup.find(id="p13n-asin-index-"+str(i))
      job_elements = book.find_all("div", class_="_cDEzb_p13n-sc-css-line-clamp-1_1Fn1y")
      price = book.find_all("span", class_="_cDEzb_p13n-sc-price_3mJ9Z")
      number_of_rating = book.find_all("span", class_="a-size-small")
      rating = book.find_all("a", class_="a-link-normal")
      book_cover = book.find_all("span", class_="a-size-small a-color-secondary a-text-normal")

      all.append(job_elements[0].text.strip())
      all.append(job_elements[1].text.strip())
      all.append(price[0].text.strip())
      all.append(number_of_rating[0].text.strip())
      all.append(rating[2].text.strip())
      all.append(book_cover[0].text.strip())

    except:
      print("--------------------------",i,"-------------------------")

    i+=1

    alls.append(all)

-------------------------- 3 -------------------------
-------------------------- 10 -------------------------
-------------------------- 15 -------------------------
-------------------------- 30 -------------------------
-------------------------- 24 -------------------------
-------------------------- 25 -------------------------
-------------------------- 30 -------------------------


In [ ]:
df = pd.DataFrame(alls,columns=['Book Name','Author','Price','Customers_Rated', 'Rating', 'Book_cover'])
df.to_csv('amazon_products.csv', index=False, encoding='utf-8')

In [ ]:
df.tail()

,Book Name,Author,Price,Customers_Rated,Rating,Book_cover
57,Why a Daughter Needs a Mom: Celebrate Your Mot...,Gregory E. Lang,$6.89,"2,349","4.9 out of 5 stars 2,349",Hardcover
58,Berserk Deluxe Volume 1,Kentaro Miura,$29.99,Kentaro Miura,"4.9 out of 5 stars 7,958",Hardcover
59,Magnolia Table,Joanna Gaines,$16.59,"30,163","4.8 out of 5 stars 30,163",Hardcover
60,The Lincoln Highway: A Novel,Amor Towles,$18.00,"45,575","4.4 out of 5 stars 45,575",Hardcover
61,None,None,None,None,None,None


##### After collecting the data, we will proceed to the next step: data cleaning. Our goal is remove broken or NaN values from our table. 

### Data Cleaning

In [ ]:
import seaborn as sns
from matplotlib import pyplot as plt
import pandas as pd

##### Checking the data

In [ ]:
df = pd.read_csv("/content/drive/MyDrive/DS/Portfolio/Books and Wings/Data/amazon_products.csv")
df.head()

,Book Name,Author,Price,Customers_Rated,Rating,Book_cover
0,Finding Me: A Memoir,Viola Davis,$18.84,314,4.8 out of 5 stars 314,Hardcover
1,Atomic Habits: An Easy & Proven Way to Build G...,James Clear,$11.98,"79,977","4.8 out of 5 stars 79,977",Hardcover
2,"This Will Not Pass: Trump, Biden, and the Batt...",Jonathan Martin,$24.99,4,4.2 out of 5 stars 4,Hardcover
3,Bill O'Reilly,NaN,NaN,NaN,NaN,NaN
4,Just Tyrus: A Memoir,Tyrus,$32.49,898,4.7 out of 5 stars 898,Hardcover


##### Six columns. Price and Rating column need be modified. We need to remove the $ character and also use split to get only the rating and not the all string. Our first decision is removing rows with NaN values.


In [ ]:
df_book = df.copy()

df_book = df_book.dropna()

In [ ]:
df_book['Price'] = df_book['Price'].map(lambda x: str(x)[1:])
df_book["Price"] = pd.to_numeric(df_book["Price"])

df_book['Customers_Rated'] = df_book['Customers_Rated'].str.replace(r'\D', '')
df_book["Customers_Rated"] = pd.to_numeric(df_book["Customers_Rated"])

df_book['Rating'] = df_book['Rating'].str[:3]
df_book["Rating"] = pd.to_numeric(df_book["Rating"])

df_book = df_book.dropna()

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:4: FutureWarning: The default value of regex will change from True to False in a future version.
  after removing the cwd from sys.path.


In [ ]:
df_book

,Book Name,Author,Price,Customers_Rated,Rating,Book_cover
0,Finding Me: A Memoir,Viola Davis,18.84,314.0,4.8,Hardcover
1,Atomic Habits: An Easy & Proven Way to Build G...,James Clear,11.98,79977.0,4.8,Hardcover
2,"This Will Not Pass: Trump, Biden, and the Batt...",Jonathan Martin,24.99,4.0,4.2,Hardcover
4,Just Tyrus: A Memoir,Tyrus,32.49,898.0,4.7,Hardcover
5,Book Lovers,Emily Henry,13.60,21.0,4.6,Paperback
6,The Mothers and Daughters of the Bible Speak: ...,Shannon Bream,17.05,786.0,4.9,Hardcover
7,Johnny the Walrus,Matt Walsh,14.18,2620.0,4.9,Board book
8,Where the Crawdads Sing,Delia Owens,9.98,221019.0,4.8,Paperback
9,"Oh, the Places You'll Go!",Dr. Seuss,8.98,36141.0,4.9,Hardcover
11,It Ends with Us: A Novel (1),Colleen Hoover,11.17,88865.0,4.7,Paperback


###### After cleaning the data, now we have a suitable data to work and get our insights. We must have in mind that our goal is indicate the best books based on this data.